# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the reciprocal of the l2-norm between the item embeddings

In [1]:
name = "ItemCFEmbed";

In [2]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

## Compute cosine correlations

In [3]:
function get_embed_matrix_outdir(power)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(power))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $power-norm"
    item_embeddings = vcat(
        [
            collect(read_params("MatrixFactorization10")["A"]')
            collect(read_params("MatrixFactorization20")["A"]')
            collect(read_params("MatrixFactorization40")["A"]')
        ],
    )

    training = get_residuals("training", residual_alphas)
    S = zeros(maximum(training.item), maximum(training.item))

    kernel = item_embeddings
    p = power
    @showprogress for j = 1:size(S)[1]
        dists = map(x -> norm(x, p), eachslice(kernel .- kernel[:, j], dims = 2))
        S[:, j] = 1 ./ dists
    end

    # only the diagonal can be Inf
    for i = 1:size(S)[1]
        for j = 1:size(S)[2]
            if S[i, j] == Inf && i != j
                S[i, j] = 1e8
            end
        end
    end

    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

In [4]:
get_embed_matrix_outdir(2)

[ Debug: 20211231 12:27:10 generating similarity matrix for 2-norm
Progress: 100%|█████████████████████████████████████████| Time: 0:01:47


"ItemCFEmbed/13228483051453548148"

## Setup hyperparameters

In [4]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_alphas = ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
item_cf_params = [
    cf_params(
        name = "ItemCFEmbed.$K",
        training_residuals = ["UserItemBiases"],
        validation_residuals = ["UserItemBiases"],
        neighborhood_type = "abs",
        S = get_embed_matrix_outdir(2),
        K = K,
        λ = [1.0, 1.0, 0.0],
    ) for K in downcast_to_int.([2^6])
];

## Train models

In [5]:
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:02:14 (33.77 ms/it)
[ Debug: 20211230 23:00:55 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.425724647295681,-0.03557662489746935,0.005098522786411392,-1.671873296042151) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.6844945986937507,0.018292308107859593,-0.472041910221106,8.23659952319139): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}

Iter     Function value   Gradient norm 
     0     1.425725e+00     1.671873e+00
 * Current step size: 1.0
 * time: 0.02054595947265625
 * g(x): [-0.03557662489746935, 0.005098522786411392, -1.671873296042151]
 * x: [1.0, 1.0, 0.0]


Progress: 100%|███████████████████████████| Time: 0:02:11 (33.07 ms/it)
[ Debug: 20211230 23:03:12 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.44312024693918,0.11191032009698479,-0.008821794188353603,0.023408324961066353) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(3.7451274244503407,5.403158399217764,-2.183199686426002,1.2920846729813185): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}

     1     1.414619e+00     7.979422e-02
 * Current step size: 0.4804335482323417
 * time: 760.0301637649536
 * g(x): [0.07979421644573056, -0.009616113849434406, 0.040497402952266365]
 * x: [1.0170922041336223, 0.997550498606981, 0.8032240198124309]


Progress: 100%|███████████████████████████| Time: 0:08:31 ( 0.13  s/it)
[ Debug: 20211230 23:22:15 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.4111588869889933,0.07044798737477104,-0.009010241090896749,0.03870981601620726) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(2.324813496720667,2.9322501474651577,-1.384044932489621,1.4095548515933125): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset

...

Progress: 100%|███████████████████████████| Time: 0:03:03 (46.29 ms/it)
[ Debug: 20211231 04:18:47 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.3677751994453862,-2.7286943065786044e-9,-1.1417755119882868e-9,-1.0385959247650462e-6) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.8933903963826209,0.3226223953638269,-0.5922423174270373,153.94366728520785): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratin

    32     1.367775e+00     4.699880e-08
 * Current step size: 1.0508258027009567
 * time: 19443.664489984512
 * g(x): [1.755567343275643e-10, -2.3530122812859196e-10, 4.699879595953172e-8]
 * x: [2.4260249592712166, 1.0307683844003546, 0.0008539625746904038]


Progress: 100%|███████████████████████████| Time: 0:02:58 (45.03 ms/it)
[ Debug: 20211231 04:28:05 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.3677751994453862,-4.243621058817539e-13,4.047111948114869e-13,-1.1041855466177472e-10) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.893390416350821,0.3226224241111469,-0.5922423348703075,153.9436230197503): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratings

    33     1.367775e+00     1.176442e-11
 * Current step size: 0.9979055395424257
 * time: 19817.030965805054
 * g(x): [-5.5824202139257366e-14, -8.897519363956742e-14, -1.1764419336836426e-11]
 * x: [2.4260249557176903, 1.0307683854950727, 0.0008539625832699918]


Progress: 100%|███████████████████████████| Time: 0:29:06 ( 0.41  s/it)
[ Info: 20211231 05:02:56 training set: RMSE NaN MAE NaN R2 NaN
[ Info: 20211231 05:03:02 validation set: RMSE 1.1695192172193607 MAE 0.8479561032276048 R2 0.19456975791403786
